In [7]:
## IMPORTS
import numpy as np
import timeit
start = timeit.default_timer()
from stage import fullPrint
from datetime import datetime, date
from stage import getStage
import pandas as pd
import yahoo_fin.stock_info as yf
import random
from datetime import timedelta
from sklearn.model_selection import train_test_split

In [8]:
nasdaqList = pd.read_pickle("stockData/tickerList.pkl")
test = ['MLAIU', 'WAFU', 'MITA', 'GEEXW', 'VIVO', 'VECT', 'CCNEP', 'MSVB',
       'TWLVU', 'ORLY', 'NOVVU', 'SGIIU', 'DCTH', 'CJJD', 'SRRA', 'ASND',
       'TRMK', 'DSACU', 'GXIIU', 'GOVX', 'FLMN', 'NTIC', 'FSEA', 'CMPX',
       'PRGS', 'KRTX', 'AOSL', 'TEDU', 'APPN', 'MCAF', 'AMAO', 'ORIC',
       'EZGO', 'GO', 'RIBT', 'AGEN', 'BTCY', 'CDK', 'RRGB', 'SLDP',
       'AVGR', 'VJET', 'NXST', 'ICMB', 'BBIO', 'XRTX', 'GLHA', 'KTRA',
       'ELTK', 'SIVBP', 'KLAQ', 'SCKT', 'SBGI', 'CHRW', 'HIFS', 'HALO',
       'MAR', 'COMS', 'LRMR', 'AZTA', 'IDBA', 'IQMD', 'GTH', 'CCOI',
       'CTSO', 'RTLR', 'CSQ', 'AWH', 'MFIN', 'HOTH', 'PRTG', 'WLFC', 'GH',
       'ASYS', 'NVIV', 'DSAC', 'UGRO', 'HCAT', 'REVB', 'RMBS', 'TCBS',
       'GLNG', 'MTBCO', 'CVAC', 'BWFG', 'KEQU', 'HTIA', 'MDGL', 'BCSA',
       'CERS', 'FCRD', 'SALM', 'IART', 'NYMT', 'TCRX', 'NTUS', 'UPC',
       'EPHYU', 'WKME', 'BEAT', 'OCFC', 'NWFL', 'GLLIW', 'JRSH', 'AGIL',
       'DISCA', 'TEKK', 'ATHX', 'SCWX', 'ZY', 'IHRT', 'IOBT', 'FMAO',
       'CRKN', 'NSEC', 'TMKR', 'ICAD', 'MODD', 'WEJO', 'NOAC', 'PCCTU',
       'GMVD', 'OXLCP', 'ORGN', 'TLMD', 'FAMI', 'PFDR', 'JFIN', 'CHEF',
       'ALTUU', 'NLIT', 'LYTS', 'NVEE', 'REED', 'GCBC', 'RICO', 'DTEA',
       'HMCOU', 'GPP', 'TMDI', 'YVR', 'IDN', 'RKDA', 'BENEU', 'LPTX',
       'SNBR', 'VLYPP', 'USAU', 'KRMD', 'VRTX', 'NEPT', 'UMBF', 'MITC',
       'FSSIU', 'SPRC', 'EXEL', 'NGMS', 'GTEC', 'HWKN', 'FNWB', 'UPWK',
       'MACUU', 'NTCT', 'TCFC', 'GOGL', 'MCAGU', 'REVH', 'RMBL', 'CTG',
       'MCAAU', 'JMACU', 'LDHAU', 'CAR', 'EYEN', 'CPAAU', 'PTEN', 'KALU',
       'ARCT', 'JYNT', 'VWE', 'RCKT', 'AGNC', 'CRZNU', 'IMNM', 'PXS',
       'HHR', 'AGNCO', 'AAOI', 'OPRT', 'GOOG', 'RENT', 'HURC', 'ISLE',
       'LION', 'CTRN', 'PBLA', 'DLPN', 'ACBAU', 'FLXS', 'MNTV', 'DCOMP',
       'GTYH', 'ACMR', 'RENO', 'MRM', 'THTX', 'HPLT', 'CURI', 'TMDX',
       'ZSAN', 'UPTDU', 'VORBW', 'ASPS', 'ARYE', 'ITRM', 'VCXAU', 'THRX',
       'NEXT', 'WWAC', 'IMPL', 'BLTSU', 'KOSS', 'MLVF', 'ARGUU', 'CRMT',
       'IMOS', 'PTIC', 'ACTDU', 'SP', 'AKAM', 'ZT', 'DTRT', 'TURN',
       'BTAI', 'SIBN', 'KVSA', 'XPAXU', 'PCH', 'KE', 'GEVO', 'OSUR',
       'MPRAW', 'FTPAU', 'ANGO', 'ATNI', 'DHAC', 'NFE', 'SWAG', 'ZI',
       'FOSL', 'CBIO', 'BNIX', 'PCSB', 'NCTY', 'WEJOW', 'LANC', 'FORR',
       'DMAC', 'RAMMU', 'GRABW', 'INOD', 'LIVE', 'CYTK', 'ROSE', 'ENPH',
       'AEACU', 'JCTCF', 'CELZ', 'ALIM', 'TCRR', 'RWAY', 'AGRI', 'VSAT',
       'IVCBU', 'ALXO', 'SLN', 'SIER', 'NRBO', 'HSDT', 'QFIN', 'DLTR',
       'MMSI', 'ESPR', 'MIME', 'ENTF', 'IMAB', 'GLDD', 'CHSCL', 'DGII',
       'TILE', 'PBAXU', 'GOBI', 'ALYA', 'LHAA', 'HTGM', 'SINT', 'CCXI',
       'SKIN', 'BNR', 'SPTK', 'CDNS', 'ETON', 'XPDBU', 'DAVEW', 'MRUS',
       'PCXCW', 'MFH', 'AUVI', 'MTEX', 'AVAH', 'AEHR', 'NMTR', 'USAP',
       'RIVN', 'NYMTM', 'LVLU', 'FSLR', 'ABVC', 'HYPR', 'ENTG', 'IRDM',
       'KZR', 'CSCW', 'GSHD', 'OYST', 'SPSC', 'INDT', 'WASH', 'TXRH',
       'VRME', 'VENA', 'FSRX', 'HBANP', 'STTK', 'CRZN', 'VRTS', 'GLPI',
       'BOKF', 'EBC', 'CMRX', 'CCAP', 'CRSR', 'DSWL', 'BJDX', 'CLBT',
       'GWRS', 'CISO', 'GLAQ', 'AVID', 'SEEL', 'CFFI', 'SREV', 'ACB',
       'TUEM', 'EDTK', 'GSBC', 'LOCO', 'CSWI', 'CNXN', 'SMSI', 'AEIS',
       'HSKA', 'GBRGU', 'WTMA', 'CLBS', 'FRPH', 'BASE', 'ZWRK', 'DWACU',
       'SCPS', 'SRSA', 'ANAB', 'FHLT', 'QNST', 'KRT', 'RMNI', 'RGNX',
       'EKSO', 'VIAV', 'VIIAU', 'ITCI', 'JAMF', 'SCMAU', 'SQL', 'OXSQ',
       'FRSGU', 'USCB', 'INDP', 'NPAB', 'UBX', 'MNTX', 'OPEN', 'AMEH',
       'ACBI', 'GAIA', 'FSBC', 'BBSI', 'ZIMVV', 'PRLHU', 'EHTH', 'CFIV',
       'HUBG', 'AMKR', 'HPK', 'AY', 'CLAQU', 'DGLY', 'TACO', 'ROSEU',
       'AROW', 'SWKS', 'BSKYU', 'AFMD', 'AGMH', 'AVGOP', 'PPBT', 'WSBF',
       'PHIC', 'WVVI', 'APTM', 'ESGR', 'VAQC', 'XFINW', 'CLAR', 'IMBI',
       'SPFI', 'PAAS', 'BTAQU', 'TCBIO', 'HMPT', 'UROY', 'APLT', 'LSXMK',
       'PEBO', 'QIWI', 'HFFG', 'BIOC', 'HQI', 'MARPS', 'APCX', 'FNCB',
       'BHFAM', 'NRC', 'TGA', 'SHIP', 'HMNF', 'STRRP', 'UK', 'GLBLU',
       'PRIM', 'XBIO', 'DISA', 'KPLT', 'ARCC', 'QLGN', 'FWONA', 'SONN',
       'APR', 'BFI', 'AVGO', 'IBCP', 'CBAT', 'KALV', 'CMLS', 'MACA',
       'MTAC', 'WKEY', 'GSKY', 'III', 'EPSN', 'BHAT', 'UTME', 'RCRT',
       'IPDN', 'COLB', 'RPTX', 'AMAL', 'LCA', 'DOOO', 'ABUS', 'GERN',
       'QUIK', 'PWFL', 'DADA', 'APDN', 'MEOH', 'GIGM', 'REKR', 'PLSE',
       'PWP', 'VENAU', 'SUNW', 'SISI', 'SHCR', 'HSIC', 'FLGT', 'IDEX',
       'LDHA', 'WW', 'CNNB', 'VIRX', 'IPWR', 'VICR', 'LOV', 'SAVA',
       'RILYL', 'LGTOU', 'AEPPZ', 'FTCV', 'FATBB', 'CRVL', 'AMRS',
       'MOBQW', 'RVSB', 'HLXA', 'BNOX', 'SBT', 'TBBK', 'CRNC', 'CRTD',
       'MGRC', 'CSSEP', 'CLSD', 'RXT', 'TGAA', 'ALKT', 'EXAI', 'FLL',
       'DTST', 'BLSA', 'LMAT', 'SPWR', 'WAVD', 'TTSH', 'MESO', 'RADI',
       'TTEC', 'HRTX', 'CYBE', 'DH', 'PSNL', 'KLAQU', 'NURO', 'FNWD',
       'DGICB', 'HEAR', 'BFRA', 'XAIR', 'MCAEU', 'EDNC', 'PFTA', 'MIRO',
       'VLDR', 'IROQ', 'MNTK', 'NCACU', 'TYME', 'TELA', 'THCP', 'GDYN',
       'APTO', 'DSKE', 'SVC', 'ARCK', 'NMRD', 'CDAQU', 'HUGE', 'CDXC',
       'POAI', 'OTEX', 'NXTD', 'SPRO', 'LGVCU', 'MGEE', 'COOL', 'PCRX',
       'DWAC', 'ALAC', 'GLG', 'FGFPP', 'AVDX', 'GLMD', 'LVTX', 'TTMI',
       'WDAY', 'CRTO', 'MSAC', 'FRBA', 'MMAT', 'MUDS', 'MCBS', 'EXPD',
       'NWL', 'CGNT', 'AVT', 'IGTAU', 'GENE', 'FRBNU', 'DSGN', 'OZK',
       'VCYT', 'AYRO', 'IESC', 'PKBK', 'GMTX', 'BRPM', 'GBNY', 'HVBC',
       'CVLY', 'KRNLU', 'ATXS', 'VRCA', 'BIOSU', 'CSIQ', 'ADVM', 'FRAF',
       'ARHS', 'TRIB', 'SMAPU', 'IPW', 'AAWW', 'LOGI', 'PGC', 'ENLV',
       'CNOB', 'CATY', 'GVP', 'ATEC', 'VNDA', 'PBBK', 'CMCTP', 'NYMTL',
       'NICE', 'ALRN', 'PTSI', 'CLOE', 'SOPH', 'AGYS', 'TW', 'LMNR',
       'POSH', 'EVLV', 'BBQ', 'ALBO', 'POWW', 'FOUNU', 'RSVR', 'ONEM',
       'CYCC', 'BRFH', 'IPAXU', 'BOCNU', 'EDSA', 'LKFN', 'GNAC', 'ZNTEU',
       'GIFI', 'KRON', 'GLBE', 'ICLR', 'BWCAU', 'STRR', 'HBANM', 'AVCO',
       'GRCY', 'YELL', 'ISIG', 'SYBX', 'CMCO', 'HCDI', 'SYNL', 'AIRTP',
       'IMCR', 'ACXP', 'NXPI', 'OBSV', 'ETNB', 'PRSO', 'SCVL', 'VYNE',
       'MTCR', 'WALDU', 'APA', 'EEIQ', 'OSPN', 'GTPBU', 'BYND', 'SSP',
       'JANX', 'OCFCP', 'KMDA', 'ARBG', 'NHTC', 'KFRC', 'TROO', 'SANA',
       'HTLD', 'AVTE', 'PPC', 'HRMY', 'MANT', 'OLITU', 'ASTE', 'DNLI',
       'FWBI', 'PFG', 'BCTX', 'WINVU', 'EQOS', 'VIGL', 'VACC', 'EQBK',
       'ODFL', 'PAHC', 'RMR', 'AMZN', 'BHAC', 'OHPA', 'APPS', 'AEHAU',
       'LSXMA', 'AEAC', 'STRA', 'OGI', 'HYACU', 'ACTD', 'NVNO', 'PTON',
       'OXBRW', 'VINC', 'TBCP', 'BENE', 'ACGL', 'SLS', 'LCAA', 'FSTR',
       'PTVE', 'CMMB', 'XTLB', 'INKT', 'IKT', 'WVE', 'MACK', 'WATT',
       'HCVIU', 'EVK', 'GTPAU', 'GRAY', 'HWBK', 'MDIA', 'NEWT', 'LAZR',
       'FATE', 'PTOCU', 'BGCP', 'CNEY', 'SNOA', 'LQDT', 'LGND', 'GLSI',
       'AKUS', 'XM', 'DNAB', 'OCCIN', 'JKHY', 'CMCT', 'SRZN', 'CHDN',
       'PGRW', 'CDZI', 'DUOL', 'ZIVO', 'AGBA', 'RCACU', 'PNRG', 'ASTR',
       'TRUE', 'ADP', 'KNTE', 'HOFV', 'INCY', 'SVRA', 'WHLR', 'CDAQW',
       'INDI', 'DFH', 'ONYX', 'VRAR', 'GECC', 'CALT', 'ONEW', 'CNTA',
       'CGO', 'LAND', 'UVSP', 'INMB', 'CDXS', 'FUSB', 'CCD', 'URGN',
       'FXNC', 'LVAC', 'PLUG', 'CSGS', 'KNSA', 'ENSG', 'PDSB', 'AFBI',
       'ECOR', 'SELB', 'SSBK', 'FIACU', 'XEL', 'CRTX', 'LPTH', 'CTIC',
       'AHI', 'SPTKU', 'DLHC', 'FANH', 'HBIO', 'PCCT', 'LAAAU', 'WSFS',
       'NPABU', 'USIO', 'SRRK', 'INZY', 'MASI', 'ARQT', 'TIVC', 'FUNC',
       'TCBK', 'MOTS', 'USEG', 'TFFP', 'BIIB', 'CHNG', 'PHAR', 'PIRS',
       'FISV', 'AVDL', 'TSBK', 'JD', 'DYNS', 'OIIM', 'MAAQU', 'KTOS',
       'DTIL', 'TITN', 'IRIX', 'ITRI', 'DJCO', 'KLAC', 'IMUX', 'HORIU',
       'QIPT', 'GDNRU', 'NVAX', 'COLM', 'SGTX', 'ADAG', 'VYGR', 'ALJJ',
       'NHIC', 'LSBK', 'DORM', 'ALTU', 'AEHA', 'MEDS', 'AEAEW', 'PXLW',
       'FSBW', 'APAC', 'LINC', 'NCSM', 'RXRX', 'TWCBU', 'BRY', 'AEP',
       'ISAA', 'MSGM', 'SVFC', 'BBIG', 'GXII', 'VTAQ', 'KELYA', 'ARTNA',
       'VEV', 'SUMO', 'MIGI', 'LTBR', 'PETS', 'ITOS', 'SWSSU', 'BDSI',
       'SSSS', 'ASPCU', 'JJSF', 'ECOL', 'SVFA', 'NXTP', 'GGPIU', 'FIVE',
       'FSTX', 'CBAY', 'CRNX', 'XRAY', 'AHCO', 'RAIL', 'POOL', 'SFBC',
       'WRAP', 'AKYA', 'RNA', 'CDAQ', 'DYAI', 'SMLR', 'OHAA', 'USWS',
       'STEP', 'KLTR', 'FIAC', 'VFF', 'AXTI', 'AACIU', 'MBWM', 'TLIS',
       'ARRY', 'ISO', 'KPTI', 'EM', 'HQY', 'GRWG', 'GPCOU', 'CFFEU',
       'AHPAU', 'BKYI', 'STOK', 'PGEN', 'LARK', 'SBFG', 'DMTK', 'MHUA',
       'WINT', 'ADGI', 'RAIN', 'PBAX', 'BNTC', 'BIOT', 'HCNE', 'CAN',
       'NTRB', 'LCAPU', 'BBI', 'RAPT', 'OCAXU', 'AOGOU', 'BON', 'FBNC',
       'EYES', 'FUTU', 'MDXG', 'AGNCN', 'MKSI', 'SURF', 'TSHA', 'CDEV',
       'ROST', 'AEI', 'RNER', 'KIDS', 'PMTS', 'VPCBU', 'VNET', 'ANIK',
       'LITE', 'SECO', 'OBLG', 'SGRP', 'HOOK', 'SAMA', 'VBFC', 'CGC',
       'EGBN', 'GSEVU', 'BRLT', 'VRRM', 'RNW', 'NATI', 'SDAC', 'IEA',
       'USAK', 'AACG', 'ALTR', 'LUNA', 'AKTX', 'DTRTU', 'ASPU', 'XENE','EPIX', 'DGICA', 'VLATU', 'ZION', 'AMCX', 'ULBI', 'PGNY', 'RGLD',
       'OTECU', 'KVSC', 'AGGRU', 'SNES', 'BPACU', 'VERU', 'CLEU', 'QNRX',
       'IOACU', 'SLM', 'NVSA', 'METX', 'BELFA', 'GIW', 'RNDB', 'CLSN',
       'DOGZ', 'RNLX', 'FITBI', 'QRVO', 'DILAU', 'BCML', 'BYRN', 'ROCK',
       'MDGS', 'REPH', 'IPHA', 'LGSTW', 'PHUN', 'PAFOU', 'CAMP', 'VHNA',
       'MOBQ', 'LOOP', 'CLVRW', 'ETSY', 'KYMR', 'BKR', 'SYPR', 'REVEU',
       'PROC', 'ACRX', 'FINM', 'PBYI', 'PUCKU', 'LJPC', 'EVOP', 'BFC',
       'AEYE', 'GATE', 'SV', 'SONO', 'VERV', 'ORPH', 'ANPC', 'IIIV',
       'HCDIP', 'TAYD', 'VRNA', 'RSSS', 'TWNK', 'TROW', 'SRTS', 'GGMC',
       'XOMAO', 'PMD', 'BREZ', 'DISCB', 'IMPPP', 'SWBI', 'EMBK', 'WLDN',
       'SIEN', 'EVGN', 'DAKT', 'WILC', 'LIFE', 'WETF', 'JMACW', 'ITQRU',
       'PBHC', 'FFBC', 'KMPH', 'ATRC', 'SRNE', 'NWLI', 'ENTX', 'DKNG',
       'SLNH', 'SABRP', 'VEON', 'XPAX', 'OP', 'UAL', 'MINM', 'ATXI',
       'ELOX', 'SVSVU', 'CALB', 'FOXWU', 'PULM', 'RDNT', 'ATHE', 'NNBR',
       'THRY', 'EGLE', 'PROG', 'CENQ', 'PT', 'GTX', 'PAFO', 'CMPO',
       'PHCF', 'PNTG', 'FOX', 'WHLRP', 'ORGS', 'LNDC', 'OPI', 'HCARU',
       'GRUB', 'UCBIO', 'HBT', 'LHCG', 'BSET', 'PPIH', 'BNTX', 'MILE',
       'SNPS', 'NESR', 'BRIV', 'CECE', 'ICCM', 'KRBP', 'OTEC', 'GLYC',
       'PCTTU', 'JBHT', 'TGAN', 'NDRA', 'NUVL', 'BZ', 'ZVO', 'BTBD',
       'SUMR', 'CCRN', 'METC', 'AZPN', 'FUSN', 'BROG', 'LMFA', 'SCOB',
       'MODV', 'BLRX', 'DENN', 'CRESY', 'GDEN', 'CYTH', 'GNACU', 'HOPE',
       'GRNQ', 'AUR', 'BIVI', 'FLAC', 'ADN', 'VLY', 'THACU', 'ACAD',
       'HNRG', 'CDMO', 'FANG', 'BIOS', 'PTRA', 'GMBLP', 'IMTX', 'OKTA',
       'NSSC', 'ZEAL', 'GRVI', 'NDLS', 'ROC', 'RESN', 'IMRN', 'EWCZ',
       'BCOV', 'THMO', 'PRTS', 'ADALU', 'AMPG', 'BLCT', 'PPGH', 'SZZLU',
       'GTPA', 'CERN', 'VRNS', 'SERA', 'MYNA', 'SRCL', 'PBFS', 'ROIC',
       'IZEA', 'SNRHU', 'FREE', 'MGI', 'PCSA', 'FITB', 'EOSE', 'NUVA',
       'FGEN', 'GVCIU', 'VCKA', 'ERYP', 'PBCTP', 'NYMTZ', 'MSDAU', 'PNFP',
       'ORIA', 'PRCH', 'HGSH', 'VSTM', 'SOHOB', 'KTCC', 'FATPW', 'ARVL',
       'KOPN', 'OXUSU', 'IPGP', 'INSE', 'SLAB', 'ARKO', 'CLBK', 'PLBY',
       'LFST', 'MELI', 'LIBY', 'CVRX', 'QRHC', 'MBIO', 'STAB', 'VIEW',
       'BFIN', 'AGTC', 'RCMT', 'IIVI', 'SBNY', 'OPRA', 'ATHA', 'FCAP',
       'GTIM', 'PFIN', 'FNLC', 'AXSM', 'ASPC', 'APCXW', 'HSAQ', 'HSTM',
       'OXACU', 'UNCY', 'CSTL', 'KINZU', 'MTSI', 'OCGN', 'ENTFW', 'ABIO',
       'SOHON', 'CSTE', 'XBIT', 'ALPAU', 'KDNY', 'PNT', 'ELDN', 'ANZU',
       'VREX', 'KDP', 'MON', 'SHCAW', 'APTX', 'PROF', 'FCCO', 'GOSS',
       'LULU', 'KRNL', 'ATY', 'IMPP', 'VAXX', 'NEOG', 'TZPS', 'NRXPW',
       'ATOM', 'KLIC', 'XLO', 'CBNK', 'TEKKU', 'BMEA', 'PSTX', 'APXIU',
       'PDD', 'UBCP', 'MCRB', 'QCOM', 'ICCC', 'NOVT', 'INTEW', 'TUGCU',
       'TCON', 'VTIQ', 'GDEV', 'TGTX', 'FRGAP', 'AUID', 'XELB', 'BLDE',
       'GLEE', 'TIG', 'LNTH', 'ACABU', 'CACC', 'BZFD', 'TWOU', 'BELFB',
       'HTIBP', 'BYTSU', 'BLPH', 'CBSH', 'AVHI', 'ISTR', 'MUDSU', 'MSBI',
       'CVV', 'TCMD', 'AKU', 'ACIU', 'BTNB', 'BKSC', 'EZFL', 'TVTX',
       'FINW', 'CSCO', 'CLRMU', 'REFI', 'GFGDU', 'LIAN', 'XPER', 'BLMN',
       'OLMA', 'THCAU', 'CFVIU', 'HWC', 'ROCC', 'TC', 'SY', 'JVA',
       'ARKOW', 'LBRDP', 'MATW', 'CING', 'AUDC', 'BECN', 'KAIIU', 'SPKBU',
       'ZUMZ', 'CINC', 'LFTRU', 'OPOF', 'HBNC', 'CONXU', 'EVLO', 'SEV',
       'WNW', 'OFS', 'BPYPP', 'BYTS', 'PFIE', 'COVAU', 'YGMZ', 'LPLA',
       'ZNTE', 'NCBS', 'RDHL', 'DUO', 'LVOX', 'GLBZ', 'CCEL', 'AQMS',
       'DRMA', 'NTRS', 'DNUT', 'DMLP', 'FLIC', 'UPLD', 'FCFS', 'MBRX',
       'LTRY', 'MGIC', 'AHPI', 'BTWNU', 'SSNT', 'ACVA', 'PIII', 'IGAC',
       'CLAY', 'ARDX', 'FRLAW', 'TCVA', 'EDRY', 'HCICU', 'BRTX', 'EVOL',
       'UBSI', 'INM', 'OPK', 'EJFAU', 'HYRE', 'CHCO', 'CWBR', 'SPCB',
       'PERI', 'TZOO', 'CELUW', 'GPRO', 'ONCY', 'XOS', 'ONYXU', 'RADA',
       'AFAQ', 'KLXE', 'ME', 'ADIL', 'NGM', 'GANX', 'FELE', 'AUBAP',
       'FFHL', 'BOXL', 'ROCAU', 'SGMO', 'SSNC', 'MACU', 'ZG', 'SWIM',
       'OPRX', 'APTMU', 'DMRC', 'WB', 'TNDM', 'GAMC', 'ANGI', 'CCNC',
       'DRUG', 'CRMD', 'ZNTL', 'OXAC', 'BIOTU', 'AINV', 'ALPP', 'OCG',
       'OVLY', 'UEIC', 'PRLH', 'IMAQU', 'RNERW', 'FTVI', 'AUVIP', 'ANNX',
       'MNDO', 'RGEN', 'BTB', 'BBBY', 'NWSA', 'MDJH', 'THFF', 'CIDM',
       'SLDB', 'OXUS', 'FRBK', 'NEPH', 'SCOA', 'RRBI', 'DYNT', 'FTRP',
       'FRSG', 'LILM', 'KSI', 'BLDP', 'HLMN', 'BLIN', 'PI', 'ADXN',
       'VMEO', 'AUBN', 'COMSP', 'TIOA', 'ITRN', 'SCOR', 'CLVR', 'MAAQ',
       'GIAC', 'CRNT', 'EEFT', 'ULH', 'HMST', 'CFFN', 'SHLS', 'MBIN',
       'BNFT', 'DXCM', 'SMTI', 'FTHM', 'TRIP', 'HCWB', 'DHHC', 'RGP',
       'IMAQ', 'FGBIP', 'ADTHW', 'BKCC', 'JWEL', 'OEPW', 'CIIGU', 'ANDE',
       'AFCG', 'SABS', 'SBFMW', 'BILI', 'SEIC', 'APWC', 'REE', 'GYRO',
       'ZIONO', 'PTIX', 'ASLN', 'STAF', 'SATS', 'LYEL', 'LYFT', 'DWSN',
       'GPCO', 'OSBC', 'MPAC', 'WDC', 'NKSH', 'CMPOW', 'LPCN', 'PINC',
       'BIOL', 'AMRK', 'HCIC', 'DSP', 'SNPX', 'CREG', 'ABMD', 'BATRK',
       'NH', 'HROW', 'CLST', 'WFCF', 'RLYB', 'BSGM', 'WVVIP', 'FOUNW',
       'CVBF', 'MBUU', 'OHPAU', 'CELU', 'XPEL', 'RCHG', 'DISH', 'BYSI',
       'MESA', 'SLMBP', 'CSX', 'PLRX', 'LIZI', 'ALPA', 'MULN', 'HFBL',
       'AIP', 'TUGC', 'NTLA', 'PEGRU', 'EVOJU', 'RUSHB', 'INTG', 'MCBC',
       'PECO', 'DUNEU', 'EXAS', 'COLI', 'ADSE', 'LWAY', 'CORT', 'SQFTW',
       'ZEST', 'CPLP', 'NBN', 'FMIV', 'GGGVU', 'ALPN', 'RNXT', 'CMPI',
       'ELMS', 'PCTI', 'GLUE', 'JZXN', 'MRKR', 'SIMO', 'SNPO', 'PSAGU',
       'PRVB', 'NMMCU', 'MBINO', 'MOFG', 'GSM', 'TIGR', 'ITIC', 'SWETU',
       'HEES', 'PLXP', 'KSICU', 'GT', 'SCMA', 'MSTR', 'IIN', 'TOAC',
       'GDRX', 'BSFC', 'INTEU', 'REG', 'KRKR', 'CEVA', 'ROLLP', 'FTPA',
       'GHSI', 'TUSK', 'VIOT', 'RNERU', 'CMBM', 'FDBC', 'FRSX', 'IGMS',
       'SRDX', 'MMLP', 'BATRA', 'FTCI', 'PODD', 'ALOR', 'EGAN', 'OZON',
       'PUCK', 'DCBO', 'NXTC', 'WKHS', 'FB', 'COHU', 'ABCB', 'SBLK',
       'LBPS', 'CRCT', 'ACGLO', 'VCSA', 'APMI', 'CLVS', 'SNGX', 'RVNC',
       'AMAOU', 'SAFT', 'QDEL', 'AYLA', 'CLGN', 'GPAC', 'AIKI', 'ETAC',
       'HSII', 'RGF', 'IMXI', 'CTAQ', 'QTNT', 'LNSR', 'GHRS', 'CPAR',
       'RPRX', 'MBII', 'IMRX', 'SPWH', 'FCEL', 'MINDP', 'CFLT', 'RNWK',
       'FIACW', 'MAPS', 'MLAI', 'HGEN', 'TCX', 'CVET', 'APEI', 'MORF',
       'LJAQU', 'HARP', 'LABP', 'CENTA', 'ASUR', 'BEEM', 'VINP', 'FSRXU',
       'BCLI', 'LBRDA', 'CIGI', 'ASTS', 'CZNC', 'SPI', 'FATP', 'FNGR',
       'FLWS', 'WING', 'HAIAU', 'RVMD', 'NDAQ', 'IVAC', 'EPZM', 'DCRD',
       'ATER', 'SSBI', 'UEPS', 'RRR', 'OPBK', 'OTLY', 'ALNY', 'STRNW',
       'MCRI', 'NCNO', 'LYL', 'PENN', 'CRIS', 'CWST', 'ATLO', 'MARK',
       'AHPA', 'IMGO', 'GRPH', 'CUEN', 'LAZY', 'CWBC', 'FTAA', 'PCYO',
       'ATAI', 'DTOC', 'LBRDK', 'BHFAO', 'STKL', 'VCKAU', 'AVAC', 'MTBCP',
       'GTACW', 'ASPA', 'LINK', 'ENTFU', 'SGMS', 'ALSAU', 'IBRX', 'KERN',
       'PEBK', 'PLUS', 'LAWS', 'SEAC', 'INSG', 'CVCO', 'DICE', 'BLI',
       'WSBCP', 'WTFC', 'ENOB', 'RGLS', 'BPAC', 'MEOAU', 'FLEX', 'PCYG',
       'SKYW', 'VIEWW', 'NVDA', 'SND', 'ATLC', 'ACT', 'AVRO', 'ASAXU',
       'BRCN', 'VWTR', 'PFLT', 'RANI', 'SGMA', 'CLMT', 'CRECU', 'BWAY',
       'CLDX', 'SOFI', 'CTXR', 'AXLA', 'SHBI', 'SFM', 'MCAFU', 'OPGN',
       'CHUY', 'AXON', 'NWS', 'VIVE', 'FEMY', 'CTLP', 'VITL', 'LVRAU',
       'BANF', 'AMYT', 'FLUX', 'UNTY', 'EH', 'TBK', 'LGTOW', 'OTRA',
       'STRT', 'PSMT', 'WKSP', 'COGT', 'TGLS', 'EQRX', 'PRQR', 'TRVI',
       'ALCO', 'GATEW', 'MTEM', 'LTRPB', 'FIXX', 'SGLB', 'ADSEW', 'LMB',
       'LASR', 'ALACW', 'VIRC', 'TMC', 'ADOC', 'CRWD', 'SPTN', 'FLGC',
       'MITK', 'BWACU', 'IOSP', 'INNV', 'TSIBU', 'CYCCP', 'OB', 'CTAQU',
       'OMGA', 'LMAOU', 'PRAX', 'SMIH', 'ATSG', 'HCCC', 'AFAC', 'MNOV',
       'RNST', 'GGAL', 'ABGI', 'DSGX', 'GBIO', 'WTFCP', 'KROS', 'CHSCO',
       'SBCF', 'CEMI', 'ACIW', 'FRLAU', 'PRTA', 'TOI', 'WTBA', 'YMAB',
       'GFED', 'GDS', 'KIIIU', 'LGACU', 'WRLD', 'SFET', 'RLMD', 'CKPT',
       'NOACU', 'HOOD', 'OM', 'ALLR', 'MYRG', 'FRONU', 'NSYS', 'MDXH',
       'ESBK', 'ALORU', 'LLL', 'VRA', 'WAFD', 'REFR', 'JYAC', 'TMKRU',
       'KTTA', 'DYN', 'CASS', 'INFN', 'ATNF', 'SBFM', 'OTRK', 'CLWT',
       'BKEP', 'HLNE', 'WYNN', 'FCNCA', 'BRP', 'BIGC', 'VTAQU', 'SLDPW',
       'ADI', 'VSAC', 'BPTS', 'CRDL', 'GSIT', 'AVACW', 'KAIR', 'BFST',
       'LGO', 'BLCM', 'MF', 'KRNY', 'VLCN', 'ASRV', 'UNB', 'GLAD', 'TRS',
       'CNSP', 'CETXP', 'UG', 'PSHG', 'XMTR', 'VERX', 'MTBC', 'FWP',
       'BUSE', 'GET', 'AATC', 'HELE', 'GIIXU', 'CFVI', 'ADTN', 'LIVN',
       'EVFM', 'BPYPN', 'ACHL', 'MAQCU', 'NBTX', 'SHQA', 'PACW', 'STSA',
       'ACLX', 'ARGX', 'VLYPO', 'SLHG', 'PYPD', 'IMGN', 'XELA', 'HUDI',
       'AMCIU', 'FA', 'FIVN', 'COIN', 'HLTH', 'BGFV', 'DBX', 'CGBD',
       'TTNP', 'PTC', 'LEXX', 'JNCE', 'STKS', 'INSM', 'ONVO', 'PEGA',
       'PLMI', 'TBSA', 'LIBYW', 'SAGAU', 'GABC', 'ARAY', 'KHC', 'ARNA',
       'XXII', 'CCBG', 'DBGI', 'LVOXU', 'BCSAU', 'CLOEU','PDEX', 'ARDS', 'OEPWU', 'ARTE', 'PEPLU', 'TRHC', 'OFIX', 'CMTL',
       'MEOA', 'VTGN', 'GWII', 'CWCO', 'CFFE', 'MIDD', 'NEON', 'ARKR',
       'BAND', 'VOXX', 'GOEV', 'GREE', 'FCNCP', 'NLITU', 'NVOS', 'LSTR',
       'IXAQ', 'LGVC', 'PDCE', 'AUB', 'ALT', 'EAC', 'GHAC', 'SMCI', 'SLP',
       'CAC', 'ALGN', 'TWST', 'UFPT', 'DGHI', 'CSII', 'RIGL', 'TWCB',
       'WHLRD', 'TXMD', 'SURG', 'KBNT', 'AFYA', 'CSTR', 'QLI', 'SLCRU',
       'VMD', 'ARCE', 'TALK', 'HIMX', 'GMBL', 'VTNR', 'EVCM', 'FUV',
       'KPRX', 'GACQ', 'SVFD', 'TKNO', 'QK', 'VXRT', 'CHK', 'FRON',
       'VMAR', 'TPST', 'GNTA', 'VTSI', 'PNNT', 'UBFO', 'HYZN', 'GEG',
       'HWEL', 'DKDCA', 'AVEO', 'PLMIU', 'CPIX', 'TAOP', 'SONM', 'CVLG',
       'SHC', 'CREC', 'AMTI', 'AAON', 'MDRRP', 'ANAT', 'RVAC', 'FRGI',
       'AWRE', 'PTICU', 'VSACU', 'BRLIU', 'EUCRU', 'PYXS', 'CCAI', 'RBKB',
       'GLBS', 'PCTY', 'OCDX', 'WULF', 'CONE', 'IGACU', 'MPACU', 'TOACU',
       'PCXCU', 'TARS', 'PLAY', 'WISH', 'ONCS', 'PEGR', 'BCEL', 'RMGC',
       'FORD', 'ORRF', 'RMBI', 'ARBGU', 'PKOH', 'LMAO', 'APMIU', 'LAMR',
       'WBA', 'ADER', 'TSCO', 'ABEO', 'QH', 'MYPS', 'SNEX', 'MMMB',
       'CASA', 'SMTC', 'MTLS', 'SWKH', 'WNEB', 'XFINU', 'FISI', 'MTRY',
       'MNST', 'FARO', 'CROX', 'OLLI', 'BVS', 'HYFM', 'CARE', 'RACB',
       'SIVB', 'HIHO', 'LTRPA', 'PBIP', 'SEER', 'RCLFU', 'NYMTN', 'TETCU',
       'LGVN', 'VIRI', 'FREQ', 'CMPR', 'FSV', 'HLBZ', 'BPTH', 'LUCD',
       'CTMX', 'ONB', 'SMMF', 'ICPT', 'DOCU', 'NHICU', 'GOODN', 'KAVL',
       'ACHC', 'APLS', 'FIZZ', 'SRAD', 'ISBC', 'FWONK', 'TRMD', 'VORB',
       'FMNB', 'ERAS', 'MCAG', 'SDH', 'OCX', 'BRKR', 'ALHC', 'RDVT',
       'LUNG', 'OMAB', 'OSTRU', 'VINO', 'PRAA', 'VTVT', 'UONEK', 'SIGIP',
       'MXCT', 'GSMGW', 'AGLE', 'SYRS', 'ITQ', 'SMED', 'JUPW', 'GTACU',
       'CASY', 'REDU', 'CNTB', 'LBPH', 'COFS', 'ACEV', 'TREE', 'NTGR',
       'MTP', 'NVFY', 'DOYU', 'WSTG', 'DHIL', 'ENERW', 'SSAA', 'SCPL',
       'LOVE', 'IMMR', 'IMMX', 'RZLT', 'SAMG', 'CLPT', 'IMTE', 'BGRY',
       'DRIO', 'BDTX', 'WSC', 'LOTZ', 'MLAC', 'HTOO', 'QLYS', 'ACET',
       'WAFDP', 'LOGC', 'SIERU', 'VERA', 'FKWL', 'EBON', 'GOOD', 'TEAM',
       'PTE', 'CRWS', 'GP', 'FNVT', 'Z', 'AEZS', 'BLUE', 'XNCR', 'AMGN',
       'IRTC', 'AMST', 'RKLB', 'CCTS', 'ARIZ', 'SEED', 'STIM', 'CMAX',
       'MTRX', 'SRGA', 'GALT', 'HOUR', 'CALA', 'EVOK', 'ALLO', 'HSON',
       'SMBC', 'FGF', 'ASMB', 'KINZ', 'ADTX', 'SLNO', 'PFTAU', 'PHIO',
       'TGVCW', 'KVHI', 'OXLCO', 'BTTX', 'VCXA', 'SEVN', 'SBTX', 'SPK',
       'CHTR', 'ELEV', 'GCMG', 'OSIS', 'SAFM', 'EXC', 'GGMCU', 'ALRM',
       'EBTC', 'ANEB', 'SZZL', 'PHVS', 'CUTR', 'INAB', 'SOHU', 'SSKN',
       'DXLG', 'MRNA', 'MIRM', 'ADPT', 'SNY', 'OPNT', 'AEMD', 'LGST',
       'CNSL', 'FOXF', 'CLNE', 'MNDY', 'SGEN', 'NDAC', 'OBNK', 'ADV',
       'WALD', 'HPLTU', 'FRST', 'ADUS', 'FFIC', 'JOFF', 'NODK', 'MGYR',
       'BWMN', 'EIGR', 'NSPR', 'HYAC', 'ATVI', 'OLB', 'WABC', 'CABA',
       'YTEN', 'LCAHU', 'WERN', 'HLG', 'HTZ', 'FIBK', 'PACXU', 'ATNX',
       'FVCB', 'UFCS', 'TSC', 'SLGG', 'AGGR', 'CIH', 'CFRX', 'NECB',
       'BLU', 'PTOC', 'GILD', 'DCGO', 'VLAT', 'TLRY', 'TECTP', 'CFMS',
       'AZYO', 'HUMA', 'TCBC', 'SHYF', 'ARRW', 'EBET', 'TAST', 'COOP',
       'HWELU', 'APM', 'ERII', 'IDRA', 'STRO', 'MPAA', 'HSTO', 'NLOK',
       'SPNE', 'JRVR', 'HMCO', 'MORN', 'MPWR', 'ASLE', 'GSEV', 'SMPL',
       'SOLO', 'MVIS', 'EUCR', 'WTW', 'RMGCU', 'NVNOW', 'MYMD', 'BLFS',
       'REGN', 'MNDT', 'EBACU', 'PUYI', 'SHACU', 'TOIIW', 'ABNB', 'HAYN',
       'VIVK', 'DIBS', 'XSPA', 'UCBI', 'ARTL', 'LSPRU', 'SFIX', 'CYXT',
       'NUWE', 'SKYT', 'XFIN', 'EBIX', 'CPSS', 'CHRS', 'LEE', 'AMSWA',
       'NMIH', 'EBAC', 'GNCA', 'FTAAU', 'ETACU', 'MRAI', 'FRLN', 'CPARU',
       'SSRM', 'CNTY', 'HUSN', 'ICCH', 'BRKL', 'ACAHU', 'ABCM', 'TSVT',
       'HAS', 'ARCB', 'HOLX', 'PLL', 'COMM', 'HURN', 'SIRI', 'MOHO',
       'TTOO', 'STRS', 'LCUT', 'INVZ', 'SMMT', 'ATIF', 'KAIRU', 'PRCT',
       'ZKIN', 'GBDC', 'LUXAU', 'CLXT', 'INGN', 'HYMCL', 'BRPMU', 'OPT',
       'PEAR', 'NAOV', 'REAL', 'TBNK', 'EMCF', 'LUXA', 'NVSAU', 'NVAC',
       'EMLDU', 'AIH', 'WTER', 'LANDO', 'COHR', 'FSSI', 'ARQQ', 'ECPG',
       'SANB', 'ADAL', 'DBVT', 'HALL', 'LCAAU', 'HGBL', 'KSPN', 'LKQ',
       'PONOU', 'ELSE', 'TPTX', 'SVFAU', 'JOAN', 'NVCR', 'NTNX', 'FATPU',
       'IQ', 'YTRA', 'SKYA', 'NCNA', 'CHX', 'RXST', 'RAM', 'APRE', 'ENDP',
       'TMCI', 'NCAC', 'HEXO', 'DERM', 'ICUI', 'RELL', 'HLIT', 'NAII',
       'AVACU', 'AMWD', 'JMAC', 'TETC', 'LXEH', 'SWAGU', 'WTRH', 'ROIV',
       'MANH', 'IBKR', 'SPKB', 'LBTYA', 'CCAIU', 'AMED', 'KURA', 'VPCB',
       'TLSA', 'SCSC', 'SANW', 'CENX', 'BPRN', 'YQ', 'EFOI', 'AGIO',
       'CIVB', 'LEDS', 'TNGX', 'PIK', 'FRTA', 'LSCC', 'AVIR', 'UDMY',
       'AADI', 'INO', 'CHWA', 'GBRG', 'BSQR', 'NMTC', 'DXPE', 'CNXC',
       'BRIVU', 'SWSS', 'AUTL', 'RILYP', 'AIRG', 'ALKS', 'SITM', 'RELY',
       'IRMD', 'VERO', 'PPHPU', 'CYRX', 'ELYM', 'TALS', 'CARV', 'MYFW',
       'CVGW', 'XP', 'SSB', 'VHNAU', 'CTBI', 'ZLAB', 'SNT', 'QRTEB',
       'EVO', 'IGNY', 'FTNT', 'GLTO', 'LTRX', 'ISPC', 'PRDO', 'HOWL',
       'BSGA', 'IREN', 'DAOOU', 'VIASP', 'POLA', 'NAVI', 'CGRN', 'OXLCM',
       'NVTS', 'LRFC', 'TSAT', 'MNKD', 'SPKAU', 'ZD', 'FCNCO', 'PRDS',
       'PCVX', 'ROCG', 'GTAC', 'HITI', 'TVTY', 'RMCF', 'APYX', 'OCCIO',
       'RDFN', 'MAQC', 'FNCH', 'ANTE', 'MIST', 'CLOV', 'AVO', 'HST',
       'TIL', 'BHSEU', 'GEOS', 'CVT', 'MTACU', 'MTRYW', 'ARTW', 'FDMT',
       'PONO', 'EACPU', 'FFIN', 'MTCH', 'LZ', 'BEAM', 'RVACU', 'BWBBP',
       'UTAAU', 'GPACU', 'SMID', 'IBEX', 'ALTO', 'CLFD', 'APPF', 'BLNG',
       'ZGNX', 'HBCP', 'AKTS', 'COLIU', 'GRIL', 'CTKB', 'RCAT', 'AFACU',
       'LGAC', 'UNAM', 'BLBX', 'TRON', 'CCLP', 'REYN', 'ADMA', 'CPRT',
       'JAQCU', 'BMRN', 'DCT', 'CREX', 'PFHD', 'NVCN', 'IONS', 'ATAX',
       'CZWI', 'LAXXU', 'BVXV', 'BSVN', 'SMFR', 'TRDA', 'CNTQ', 'ALLT',
       'ACBA', 'ANSS', 'XCUR', 'RELI', 'MITAU', 'CAMT', 'SCLE', 'LFTR',
       'BRID', 'QMCO', 'ORTX', 'SLGL', 'HAPP', 'TERN', 'LSEA', 'OCCI',
       'VIA', 'PBCT', 'ITI', 'DRRX', 'MGPI', 'GTHX', 'CINF', 'LFMD',
       'BNSO', 'MIND', 'EWTX', 'LUMO', 'TTWO', 'QUMU', 'AHG', 'SSTI',
       'ARBK', 'NWPX', 'DFFN', 'ROSEW', 'REAX', 'TTD', 'CELH', 'SBUX',
       'MRCC', 'CDTX', 'ICVX', 'APPH', 'RILY', 'REVHU', 'SFNC', 'LFVN',
       'FTVIU', 'SNAX', 'MNMD', 'OMEX', 'AMBA', 'ROKU', 'BSKY', 'AREC',
       'COVA', 'PRSR', 'NRSN', 'SAGA', 'SUNS', 'PRTK', 'TSCAP', 'HOLI',
       'KXIN', 'OPCH', 'NVVE', 'ASAX', 'CLSK', 'VKTX', 'BHFAN', 'CHI',
       'OTMOW', 'LMST', 'OLK', 'ANY', 'SENEB', 'LYRA', 'SGHT', 'CATC',
       'LEGA', 'ABOS', 'BZFDW', 'WVFC', 'LCID', 'VC', 'INVO', 'FNVTU',
       'DISCK', 'OTTR', 'LMACA', 'LEVLP', 'AUPH', 'DAIO', 'JOUT', 'CONX',
       'PLXS', 'PTGX', 'NVMI', 'DWACW', 'ELYS', 'PFDRU', 'HHGC', 'ROOT',
       'ONTX', 'SLNG', 'BFRI', 'UPST', 'TRONU', 'BWB', 'PRFX', 'HNST',
       'BIMI', 'BHSE', 'CTIB', 'GNTY', 'RAND', 'SPPI', 'OXBR', 'NSTG',
       'LHDX', 'ANGN', 'VELO', 'TTGT', 'ZTAQU', 'XFOR', 'GENC', 'ESQ',
       'AMPL', 'IDAI', 'NDACU', 'HLAH', 'JAN', 'CCNE', 'IGIC', 'CYCN',
       'JCS', 'EYE', 'IPA', 'WIMI', 'VOR', 'CCEP', 'BITF', 'LIQT', 'CYAD',
       'CPOP', 'GRTX', 'LNT', 'PPTA', 'OLPX', 'AMAT', 'ZNGA', 'CCSI',
       'WORX', 'SVNAU', 'ASTC', 'BPOP', 'DBTX', 'ITHXU', 'JUGGU', 'RUN',
       'LIONU', 'AYTU', 'MYGN', 'SBET', 'LYLT', 'NEXI', 'MSDA', 'DNAD',
       'TIGO', 'PEP', 'MRVI', 'MOXC', 'PPSI', 'SIOX', 'QTRX', 'HTHT',
       'FROG', 'PRFT', 'TGAAU', 'AGNCM', 'TMUS', 'FATBP', 'DNAC', 'BCAB',
       'LXRX', 'OMEG', 'HCAR', 'TBCPU', 'AIHS', 'RBB', 'BBLG', 'LESL',
       'AVCT', 'EMKR', 'IGAP', 'SRCE', 'XGN', 'LSXMB', 'EGRX', 'DRVN',
       'SIEB', 'SWIR', 'EVER', 'PRLD', 'BCDA', 'XENT', 'SNFCA', 'GTBP',
       'IRWD', 'HMTV', 'MCAA', 'RBCN', 'OVBC', 'GOCO', 'AQB', 'CSBR',
       'AMTB', 'ISSC', 'INTZ', 'SILK', 'SGC', 'SOFO', 'FTDR', 'PFIS',
       'USCTU', 'XRX', 'EBAY', 'DALN', 'GBT', 'SQFT', 'UCTT', 'UFPI',
       'KOD', 'EVBG', 'KINS', 'OPTN', 'ARBEW', 'EYPT', 'NTRSO', 'CBRG',
       'RETA', 'SGML', 'ESTA', 'KAII', 'RDBX', 'ENSC', 'UIHC', 'CFIVU',
       'CHY', 'ATRA', 'FRME', 'SNDX', 'CMCSA', 'TGVC', 'MKD', 'CLIR',
       'PLAB', 'MRNS', 'SGA', 'BIRD', 'AKIC', 'FBIOP', 'AFIB', 'LVRA',
       'IOAC', 'AIRT', 'CARG', 'GGPI', 'MRVL', 'OLED', 'EDUC', 'TA',
       'IOVA', 'TGVCU', 'SYBT', 'TBLA', 'LMNL', 'NKLA', 'VABK', 'FCUV',
       'LAUR', 'TATT', 'ICHR', 'CHCI', 'LAX', 'CTXS', 'LIXT', 'STRL',
       'CFB', 'FFIV', 'MOVE', 'AEAE', 'ADALW', 'TBPH', 'FBIZ', 'CENT',
       'THCA', 'DTOCU', 'HEPS', 'TCRT', 'SLGCW', 'HRZN', 'ORIAU', 'AKICU',
       'FRG', 'VMGAU', 'AIRS', 'ARVN', 'AVAV', 'GLLIU', 'EXTR', 'ZYNE',
       'TIOAU', 'PMGMU', 'CBRGW', 'CIIG', 'VWEWW', 'ATLCP', 'GIIX',
       'DISAU', 'LMDX', 'GIWWU', 'OVID', 'MNPR', 'MBCN', 'FOXW', 'CRECW',
       'PLYA', 'GILT', 'CBFV', 'FRW', 'MCHP', 'PAVM', 'DPCS', 'TESS',
       'PESI', 'VBLT', 'ORGO', 'BTBT']
train = []
def calculateGroupReturn(list):

    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn.to_csv("transactionTemplate.csv")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for symbol in list:
        
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
            if df == []:
                continue
        except:
            continue
        if not (df['close'] != 0).all():
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = df.index[0].strftime("%Y:%m")
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = element[0]
                lastPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            lastPrice = element[0]
    return monthlyReturn
            
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning) 
i = 0
listOfDf = calculateGroupReturn(nasdaqList)

In [9]:
indexNum = 0
sum = [1,1]
for i in range(2):
    if i == 0:
        l = train
    else:
        l = test
    index = 0
    while index != len(l):
        if l[index] not in listOfDf.columns:
            l = np.delete(l, index)
        else:
            index += 1
    for index, element in listOfDf[l].iterrows():
        #print(element.to_list())
        listOfStockRet = element.to_list()
        while 1.0 in listOfStockRet:
            listOfStockRet.remove(1.0)
        if len(listOfStockRet) != 0:
            sum[i] = sum[i] * np.mean(listOfStockRet)     
for i in range(2):
    sum[i] = sum[i] / sum[1]
print(sum)

[1.0, 1.0]


In [10]:
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
## CONSTANTS
HOLDING = 0
TOTAL = 1
def evalOneMax(params):
    print(params)
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    sectorDfList = []
    for sector in SectorDict.values():
        sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
        goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
        goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
        goodSectorDf.index = sectorDfList[0].index
        index = 0
        for date in sectorDfList[0].index:
            listOfSector = []
            for df in sectorDfList:
                try:
                    # print(df.at[date,'MA30Slope'],df.at[date,'RS'])
                    # print(individual[0][9],individual[0][10])
                    if  df.at[date,'MA30Slope'] > params[9] or df.at[date,'RS'] > params[10]:
                        if df.at[date,'close'] > df.at[date,'MA30']:
                            # print(df.at[date,'ticker'])
                            listOfSector.append(df.at[date,'ticker'])
                except: 
                    pass
                # print(listOfSector)
            goodSectorDf.iat[index,0] = set(listOfSector)
            index = index + 1
    transactionFit = pd.read_pickle("transactionTemplate.pkl")
    transactionFit['holding'] = np.empty((len(transactionFit), 0)).tolist()
    #print("Reach #1")
    tickers = pd.read_pickle('stockData/tickerList.pkl')
    for symbol in test:
        df = getStage(symbol,params,goodSectorDf)
        buyDf = pd.read_pickle('stockData/nyseNasdaq/'+symbol+'Buy.pkl')
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            open = 0
            monday = index + timedelta(3)
            i = transactionFit.index.get_loc(index)
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                if monday in buyDf.index:
                    open = buyDf.at[monday,'open']
                else:
                    open = element.close
                #delete
                transactionFit.iat[int(i),HOLDING].append((symbol,open,0))
                if buyTwice:
                    transactionFit.iat[i, HOLDING].append((symbol,open,0))
                if element.Stage == "Buy" and inStage:
                    transactionFit.iat[i, HOLDING].append((symbol,open,0))
                    buyTwice = True
                inStage = True
                continue
            if "Sell" == element.Stage[0:4]:
                if monday in buyDf.index:
                    open = buyDf.at[monday,'open']
                else:
                    open = element.close
                transactionFit.iat[i, HOLDING].append((symbol,open,-1))
                if buyTwice:
                    transactionFit.iat[i, HOLDING].append((symbol,open,-1))
                inStage = False
                buyTwice = False
    #print("Reach #2")
    total = 100
    first = True
    transactionFitCopy = transactionFit
    transactionFitCopy['total'] = 0
    #print("Reach #3")
    for index, element in transactionFitCopy.iterrows():
        indexNum = transactionFitCopy.index.get_loc(index)
        if first:
            first = False
            transactionFitCopy.iat[indexNum,1] = 100
            # print(i,transactionFitCopy.iat[i,1],"#1")
            continue
        if len(element.holding) == 0:
            transactionFitCopy.iat[indexNum,1] = transactionFitCopy.iat[indexNum-1,1]
            # print(i,transactionFitCopy.iat[i,1],"#2")
            continue
        else:
            prevData = transactionFitCopy.iat[indexNum-1, 0]
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionFitCopy.iat[indexNum,1] = total
                # print(i,transactionFitCopy.iat[indexNum,1],"#3")
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionFitCopy.iat[indexNum,1] = transactionFitCopy.iat[indexNum-1,1]
                # print(i,transactionFitCopy.iat[i,1],"#4")
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
    stockHolding  = 0
    for i in transactionFitCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    if stockHolding/(transactionFitCopy.shape[0])<(len(tickers)/1000) or transactionFitCopy.iat[-1,TOTAL]<=100:
        print(stockHolding)
        print(transactionFitCopy.iloc[-1]['total'])
        return -1
    return transactionFitCopy.iloc[-1]['total']

In [11]:
evalOneMax([0.10684256968972883, 1.1421709856192384, 0.27350824408318475, 0.9877448522578813, 1.0760547872425865, 1.2745231615209662, 0.9038554056254413, 1.0764846753295985, 1.2709398376771783, 0.992549883242513, -0.3044455098712273, 0.9467020070162686, 1.0014235121991226, 0.8164237632733558, 0.9867262709280287]
)

[0.10684256968972883, 1.1421709856192384, 0.27350824408318475, 0.9877448522578813, 1.0760547872425865, 1.2745231615209662, 0.9038554056254413, 1.0764846753295985, 1.2709398376771783, 0.992549883242513, -0.3044455098712273, 0.9467020070162686, 1.0014235121991226, 0.8164237632733558, 0.9867262709280287]


83821.670154341